# Matching Reads to Reference Sequences

## Class Trie

In [1]:
class Trie:
    
    def __init__(self):
        self.nodes = { 0:{} } # dictionary
        self.num = 0
    
    def print_trie(self):
        for k in self.nodes.keys():
            print (k, "->" , self.nodes[k]) 
    
    def add_node(self, origin, symbol):
        self.num += 1
        self.nodes[origin][symbol] = self.num
        self.nodes[self.num] = {}
    
    def add_pattern(self, p):
        pos = 0
        node = 0
        while pos < len(p):
            if p[pos] not in self.nodes[node].keys():
                self.add_node(node, p[pos])     
            node = self.nodes[node][p[pos]]
            pos += 1
            
    def trie_from_patterns(self, pats):
        for p in pats:
            self.add_pattern(p)
            
    def prefix_trie_match(self, text):
        pos = 0
        match = ""
        node = 0
        while pos < len(text):
            if text[pos] in self.nodes[node].keys():
                node = self.nodes[node][text[pos]]
                match += text[pos]
                if self.nodes[node] == {}: 
                    return match
                else:
                    pos += 1   
            else: return None
        return None
        
    def trie_matches(self, text):
        res = []
        for i in range(len(text)):
            m = self.prefix_trie_match(text[i:])
            if m != None: res.append((i,m))
        return res
        
          
def test():
    patterns = ["GAT", "CCT", "GAG"]
    t = Trie()
    t.trie_from_patterns(patterns)
    t.print_trie()

   
def test2():
    patterns = ["AGAGAT", "AGC", "AGTCC", "CAGAT", "CCTA", "GAGAT", "GAT", "TC"]
    t = Trie()
    t.trie_from_patterns(patterns)
    print (t.prefix_trie_match("GAGATCCTA"))
    print (t.trie_matches("GAGATCCTA"))
    
test()
print()
test2()

0 -> {'G': 1, 'C': 4}
1 -> {'A': 2}
2 -> {'T': 3, 'G': 7}
3 -> {}
4 -> {'C': 5}
5 -> {'T': 6}
6 -> {}
7 -> {}

GAGAT
[(0, 'GAGAT'), (2, 'GAT'), (4, 'TC'), (5, 'CCTA')]


## Class BWT

In [2]:

class BWT:
    
    def __init__(self, seq = "", buildsufarray = False):
        self.bwt = self.build_bwt(seq, buildsufarray) 
    
    def set_bwt(self, bw):
        self.bwt = bw

    def build_bwt(self, text, buildsufarray = False):
        ls = []
        for i in range(len(text)):
            ls.append(text[i:]+text[:i])
        ls.sort()
        res = ""
        for i in range(len(text)):
            res += ls[i][len(text)-1]
        if buildsufarray:
            self.sa = []
            for i in range(len(ls)):
                stpos = ls[i].index("$")
                self.sa.append(len(text)-stpos-1)
        print(self.sa)
        return res    
    
    def inverse_bwt(self):
        firstcol = self.get_first_col()
        res = ""
        c = "$" 
        occ = 1
        for i in range(len(self.bwt)):
            pos = find_ith_occ(self.bwt, c, occ)
            c = firstcol[pos]
            occ = 1
            k = pos-1
            while firstcol[k] == c and k >= 0: 
                occ += 1
                k -= 1
            res += c         
        return res        
 
    def get_first_col (self):
        firstcol = []
        for c in self.bwt: firstcol.append(c)
        firstcol.sort()  
        return firstcol
        
    def last_to_first(self):
        res = []
        firstcol = self.get_first_col()
        for i in range(len(firstcol)):
            c = self.bwt[i]
            ocs = self.bwt[:i].count(c) + 1
            res.append(find_ith_occ(firstcol, c, ocs))
        return res

    def bw_matching(self, patt):
        lf = self.last_to_first()
        res = []
        top = 0
        bottom = len(self.bwt)-1
        flag = True
        while flag and top <= bottom:
            if patt != "":
                symbol = patt[-1]
                patt = patt[:-1]
                lmat = self.bwt[top:(bottom+1)]
                if symbol in lmat:
                    topIndex = lmat.index(symbol) + top
                    bottomIndex = bottom - lmat[::-1].index(symbol)
                    top = lf[topIndex]
                    bottom = lf[bottomIndex]
                else: flag = False
            else: 
                for i in range(top, bottom+1): res.append(i)
                flag = False            
        return res        
 
    def bw_matching_pos(self, patt):
        res = []
        matches = self.bw_matching(patt)
        for m in matches:
            res.append(self.sa[m])
        res.sort()
        return res
 
# auxiliary
 
def find_ith_occ(l, elem, index):
    j,k = 0,0
    while k < index and j < len(l):
        if l[j] == elem:
            k = k +1
            if k == index: return j
        j += 1 
    return -1 


      
def test():
    seq = "TAGACAGAGA$"
    bw = BWT(seq, True)
    print (bw.bwt)
    print (bw.last_to_first())
    print (bw.bw_matching("AGA"))
    print("Suffix array:", bw.sa)
    print(bw.bw_matching_pos("AGA"))

def test2():
    bw = BWT()
    bw.set_bwt("ACG$GTAAAAC")
    print (bw.inverse_bwt())

test()

[10, 9, 3, 7, 1, 5, 4, 8, 2, 6, 0]
AGGGTCAAAA$
[1, 7, 8, 9, 10, 6, 2, 3, 4, 5, 0]
[3, 4, 5]
Suffix array: [10, 9, 3, 7, 1, 5, 4, 8, 2, 6, 0]
[1, 5, 7]


## SuffixTree

In [4]:
class SuffixTree:
    
    def __init__(self):
        self.nodes = { 0:(-1,{}) } # root node
        self.num = 0
    
    def print_tree(self):
        for k in self.nodes.keys():
            if self.nodes[k][0] < 0:
                print (k, "->", self.nodes[k][1]) 
            else:
                print (k, ":", self.nodes[k][0])
                
    def add_node(self, origin, symbol, leafnum = -1):
        self.num += 1
        self.nodes[origin][1][symbol] = self.num
        self.nodes[self.num] = (leafnum,{})
        
    def add_suffix(self, p, sufnum):
        pos = 0
        node = 0
        while pos < len(p):
            if p[pos] not in self.nodes[node][1].keys():
                if pos == len(p)-1:
                    self.add_node(node, p[pos], sufnum)
                else:
                    self.add_node(node, p[pos])     
            node = self.nodes[node][1][p[pos]]
            pos += 1
    
    def suffix_tree_from_seq(self, text):
        t = text+"$"
        for i in range(len(t)):
            self.add_suffix(t[i:], i)
            
    def find_pattern(self, pattern):
        pos = 0
        node = 0
        for pos in range(len(pattern)):
            if pattern[pos] in self.nodes[node][1].keys():
                node = self.nodes[node][1][pattern[pos]]
                pos += 1
            else: return None
        return self.get_leafes_below(node)
        

    def get_leafes_below(self, node):
        res = []
        if self.nodes[node][0] >=0: 
            res.append(self.nodes[node][0])            
        else:
            for k in self.nodes[node][1].keys():
                newnode = self.nodes[node][1][k]
                leafes = self.get_leafes_below(newnode)
                res.extend(leafes)
        return res
     
    ## exercise 1- chapter 16
    def find_parent(self, node):
        for k in self.nodes.keys():  
            if node in self.nodes[k][1].values():
                return k
        return None    
    
    def path_to_node (self, node):
        current = node
        res = ""
        while current != 0: ## root
            parent = self.find_parent(current)
            keys_parent = list(self.nodes[parent][1].keys())
            for kp in keys_parent:
                if self.nodes[parent][1][kp] == current: 
                    symb_par = kp
                    break
            res = symb_par + res
            current = parent
        return res
    
    def find_nodes_sizeK(self, k):
        active_nodes = [0]
        while (k>0):
            next_nodes = []
            for n in active_nodes:
                next_nodes.extend(list(self.nodes[n][1].values()))
            k = k - 1
            active_nodes = next_nodes
        return active_nodes
    
    def repeats(self, k, ocs):
        res = []
        active_nodes = self.find_nodes_sizeK(k)
        for n in active_nodes:
            ocs_pat = self.get_leafes_below(n)
            if len(ocs_pat) >= ocs:
                res.append(self.path_to_node(n))
        return res

def test():
    seq = "TACTA"
    st = SuffixTree()
    st.suffix_tree_from_seq(seq)
    st.print_tree()
    print (st.find_pattern("TA"))
    print (st.find_pattern("ACG"))

def test2():
    seq = "TACTA"
    st = SuffixTree()
    st.suffix_tree_from_seq(seq)
    print (st.find_pattern("TA"))
    print(st.repeats(2,2))

test()
print()
test2()
        

0 -> {'T': 1, 'A': 7, 'C': 12, '$': 18}
1 -> {'A': 2}
2 -> {'C': 3, '$': 16}
3 -> {'T': 4}
4 -> {'A': 5}
5 -> {'$': 6}
6 : 0
7 -> {'C': 8, '$': 17}
8 -> {'T': 9}
9 -> {'A': 10}
10 -> {'$': 11}
11 : 1
12 -> {'T': 13}
13 -> {'A': 14}
14 -> {'$': 15}
15 : 2
16 : 3
17 : 4
18 : 5
[0, 3]
None

[0, 3]
['TA']
